## Exercise 2
* 312441

In [1]:
# This is required to run everytime because sometimes, 
# without running it it fails to establish connection back to python.
import findspark
import matplotlib.pyplot as plt
findspark.init()
findspark.find()

'C:\\Spark\\spark-3.3.0-bin-hadoop3'

In [2]:
# Required Libraries
import pyspark
from pyspark import SparkContext, SparkConf
import numpy as np
import pandas as pd
from pyspark.sql.functions import mean , stddev, col, to_date, date_format
import pyspark.sql.functions as F
from time import strptime
import datetime
from ast import literal_eval
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.window import Window
from pyspark.sql.types import TimestampType
from pyspark.sql.functions import*

# Spark configuration and reading + printing of tags data from movielens
conf = pyspark.SparkConf().setAppName('ex8_2').setMaster('local')
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession(sc)
tags_df = sc.textFile('tags.dat').map(lambda x: x.split("::")).collect()
rdd1 = sc.parallelize(tags_df).toDF()
custom_tags = rdd1.withColumnRenamed("_1", "UserID").withColumnRenamed("_2", "MovieID").withColumnRenamed("_3", "Rating").withColumnRenamed("_4", "Timestamp")
custom_tags.show()

+------+-------+--------------------+----------+
|UserID|MovieID|              Rating| Timestamp|
+------+-------+--------------------+----------+
|    15|   4973|          excellent!|1215184630|
|    20|   1747|            politics|1188263867|
|    20|   1747|              satire|1188263867|
|    20|   2424|     chick flick 212|1188263835|
|    20|   2424|               hanks|1188263835|
|    20|   2424|                ryan|1188263835|
|    20|   2947|              action|1188263755|
|    20|   2947|                bond|1188263756|
|    20|   3033|               spoof|1188263880|
|    20|   3033|           star wars|1188263880|
|    20|   7438|              bloody|1188263801|
|    20|   7438|             kung fu|1188263801|
|    20|   7438|           Tarantino|1188263801|
|    21|  55247|                   R|1205081506|
|    21|  55253|               NC-17|1205081488|
|    25|     50|        Kevin Spacey|1166101426|
|    25|   6709|         Johnny Depp|1162147221|
|    31|     65|    

In [4]:
# 1.
# Here we first sort the data with timestamp and then add temporary column which just tells us in if
# the user meets our duration condition and later we use this columns data to generate sessionId and get rid of temp column.
user_timestamp = Window.partitionBy("UserID").orderBy("Timestamp")

custom1 = custom_tags.withColumn("isGreaterDuration", when(((col("Timestamp") - lag(col("Timestamp"), 1).over(user_timestamp)) > 1800), 1).when(((col("Timestamp") - lag(col("Timestamp"), 1).over(user_timestamp)) <= 1800), 0).otherwise(1))
custom1 = custom1.withColumn('SessionID', (sum('isGreaterDuration').over(user_timestamp)).cast("bigint"))
custom1 = custom1.drop('isGreaterDuration')
print('Users seperate tag session: ')
custom1.show()

Users seperate tag session: 
+------+-------+----------------+----------+---------+
|UserID|MovieID|          Rating| Timestamp|SessionID|
+------+-------+----------------+----------+---------+
|  1000|    277|children's story|1188533111|        1|
|  1000|   1994|    sci-fi. dark|1188533136|        1|
|  1000|   5377|         romance|1188533150|        1|
|  1000|   7147|    family bonds|1188533161|        1|
|  1000|    362|animated classic|1188533171|        1|
|  1000|    276|          family|1188533235|        1|
| 10003|  42013|        Passable|1150432435|        1|
| 10003|  51662|  FIOS on demand|1207953326|        2|
| 10003|  54997|  FIOS on demand|1207953335|        2|
| 10003|  55765|  FIOS on demand|1207953342|        2|
| 10003|  55363|  FIOS on demand|1207953420|        2|
| 10003|  56152|  FIOS on demand|1207953526|        2|
| 10003|  55116|  FIOS on demand|1207953636|        2|
| 10003|  56174|  FIOS on demand|1207953670|        2|
| 10003|  55176|  FIOS on demand|120

In [5]:
# 2. 
# I simply use groupby on my previous df and sort it by sessionid generated earlier. Further use the count function
# to finally get the frequency and add it to the df.
custom2 = custom1.groupBy(custom1.UserID, custom1.SessionID).count().orderBy(['UserID', 'SessionID'], ascending = True)

custom2 = custom2.withColumnRenamed("count", "frequency")
print('Frequency of tagging per user session: ')
custom2.show()

Frequency of tagging per user session: 
+------+---------+---------+
|UserID|SessionID|frequency|
+------+---------+---------+
|  1000|        1|        6|
| 10003|        1|        1|
| 10003|        2|       18|
| 10003|        3|       38|
| 10020|        1|        2|
| 10025|        1|        1|
| 10032|        1|       39|
| 10032|        2|        1|
| 10032|        3|        1|
| 10032|        4|        1|
| 10032|        5|        4|
| 10032|        6|        1|
| 10032|        7|        1|
| 10032|        8|        4|
| 10032|        9|        1|
| 10032|       10|        1|
| 10032|       11|        1|
| 10032|       12|        1|
| 10051|        1|        1|
| 10058|        1|       35|
+------+---------+---------+
only showing top 20 rows



In [6]:
# 3. 
# We start by first sorting with userid and then using frequency column just calculate the mean and std_dev
# using the built-in functions and save them in new columns which are then printed.
user_usergroup = Window.partitionBy("UserID").orderBy("UserID")

custom3 = custom2.withColumn('users_mean', (mean('frequency').over(user_usergroup)).cast("float")).orderBy(['UserID', 'SessionID'], ascending = True)
custom3 = custom3.withColumn('users_stddev', (stddev('frequency').over(user_usergroup)).cast("float")).orderBy(['UserID', 'SessionID'], ascending=True).replace(float('nan'), 0)
print('Each user mean and std dev: ')
custom3.show()

Each user mean and std dev: 
+------+---------+---------+----------+------------+
|UserID|SessionID|frequency|users_mean|users_stddev|
+------+---------+---------+----------+------------+
|  1000|        1|        6|       6.0|        null|
| 10003|        1|        1|      19.0|    18.52026|
| 10003|        2|       18|      19.0|    18.52026|
| 10003|        3|       38|      19.0|    18.52026|
| 10020|        1|        2|       2.0|        null|
| 10025|        1|        1|       1.0|        null|
| 10032|        1|       39| 4.6666665|   10.873933|
| 10032|        2|        1| 4.6666665|   10.873933|
| 10032|        3|        1| 4.6666665|   10.873933|
| 10032|        4|        1| 4.6666665|   10.873933|
| 10032|        5|        4| 4.6666665|   10.873933|
| 10032|        6|        1| 4.6666665|   10.873933|
| 10032|        7|        1| 4.6666665|   10.873933|
| 10032|        8|        4| 4.6666665|   10.873933|
| 10032|        9|        1| 4.6666665|   10.873933|
| 10032|       10

In [7]:
# 4.
# In this case we have to find mean/stdev across users so not limited to each user and for that i take 
# mean/stddev of frequencies. Then just print the result
mean_stddev_res = custom3.select(
    mean(col('frequency')).alias('mean_f'),
    stddev(col('frequency')).alias('stdev_f')
).collect()
np_mean, np_stdev = np.around(mean_stddev_res[0]['mean_f'], 3), np.around(mean_stddev_res[0]['stdev_f'], 3)
print('Mean across users: ', np_mean)
print('Std dev accross users: ', np_stdev)

Mean across users:  7.3
Std dev accross users:  22.264
